### Classify t4 and category by percentage change and day window values

In [1]:
# Classify t4 and category

import pandas as pd
from scipy.signal import savgol_filter
import numpy as np

# Load the dataset
dataset = pd.read_csv('Datasets/dataset_mobility.csv')

grouped_df = dataset.groupby(['state', 'mobility_type']).agg({
    'x_time': list,
    'x_date': list,
    'y_ridership_change': list,
    'phase': list,
}).reset_index()

dataset = grouped_df


def find_steady_state(y_ridership_change, threshold, window):
    """Find steady state based on raw data without smoothing."""
    consecutive_count = 0
    for i in range(75, len(y_ridership_change) - 1):
        # Small term added to avoid zero division
        change = (y_ridership_change[i] - y_ridership_change[i-1]
                  ) / (y_ridership_change[i-1] + 1e-10)
        if abs(change) <= threshold:
            consecutive_count += 1
            if consecutive_count >= window:
                return i - window + 1  # Return the starting point of the steady state
        else:
            consecutive_count = 0
    return None


def process_trajectory(row, thresholds=[0.01, 0.05], windows=[5, 7, 14]):
    y_changes = list(row['y_ridership_change'])
    smoothed_y_changes = savgol_filter(
        y_changes, window_length=25, polyorder=3)

    t4_values = {}
    for threshold in thresholds:
        for window in windows:
            t4_key = f't4_{int(threshold*100)}pct_{window}days'
            t4_values[t4_key] = find_steady_state(
                smoothed_y_changes, threshold, window)

    return pd.Series(list(t4_values.values()))


new_columns = []
thresholds = [0.01, 0.05]
windows = [5, 7, 14]
for threshold in thresholds:
    for window in windows:
        new_columns.append(f't4_{int(threshold*100)}pct_{window}days')

dataset[new_columns] = dataset.apply(process_trajectory, axis=1)


def determine_category(value):
    if value > 0.05:
        return "PG"
    elif value < -0.05:
        return "PL"
    else:
        return "PN"


for col in new_columns:
    dataset[f"{col}_category"] = dataset.apply(lambda row: determine_category(
        row['y_ridership_change'][int(row[col])] if not pd.isna(row[col]) else 0), axis=1)
result_df = dataset.groupby('mobility_type').size(
).reset_index(name='count').drop('count', axis=1)


for col in new_columns:
    category_counts = dataset.groupby(
        ['mobility_type', f"{col}_category"]).size().unstack().fillna(0)
    category_relative_percentages = (
        category_counts.T / category_counts.sum(axis=1)).T * 100

    result_df = pd.merge(result_df, category_relative_percentages.reset_index(
    ), on='mobility_type', how='left')

    for category in ["PG", "PL", "PN"]:
        # Calculate the average magnitude for this category
        avg_mag = dataset[dataset[f"{col}_category"] == category].apply(lambda row: abs(
            row['y_ridership_change'][int(row[col])] if not pd.isna(row[col]) else 0), axis=1).mean()
        result_df[f"{col}_avg_magnitude_{category}"] = avg_mag

result_df.fillna(0, inplace=True)

#result_df.to_csv('result.csv', index=False)
#dataset = dataset[dataset['t4_5pct_7days_category'] == 'PL']

dataset1 = dataset
print(np.array(dataset1['state']))
dataset1 = dataset1[dataset1['state'] != 'US']
dataset1


['Alabama' 'Alabama' 'Alabama' 'Alabama' 'Alabama' 'Alabama' 'Alaska'
 'Alaska' 'Alaska' 'Alaska' 'Alaska' 'Alaska' 'Arizona' 'Arizona'
 'Arizona' 'Arizona' 'Arizona' 'Arizona' 'Arkansas' 'Arkansas' 'Arkansas'
 'Arkansas' 'Arkansas' 'Arkansas' 'California' 'California' 'California'
 'California' 'California' 'California' 'Colorado' 'Colorado' 'Colorado'
 'Colorado' 'Colorado' 'Colorado' 'Connecticut' 'Connecticut'
 'Connecticut' 'Connecticut' 'Connecticut' 'Connecticut' 'Delaware'
 'Delaware' 'Delaware' 'Delaware' 'Delaware' 'Delaware'
 'District of Columbia' 'District of Columbia' 'District of Columbia'
 'District of Columbia' 'District of Columbia' 'District of Columbia'
 'Florida' 'Florida' 'Florida' 'Florida' 'Florida' 'Florida' 'Georgia'
 'Georgia' 'Georgia' 'Georgia' 'Georgia' 'Georgia' 'Hawaii' 'Hawaii'
 'Hawaii' 'Hawaii' 'Hawaii' 'Hawaii' 'Idaho' 'Idaho' 'Idaho' 'Idaho'
 'Idaho' 'Idaho' 'Illinois' 'Illinois' 'Illinois' 'Illinois' 'Illinois'
 'Illinois' 'Indiana' 'Indiana' 'Indi

C:\Users\Eddie\AppData\Local\Temp\ipykernel_28124\3171826867.py:83: FutureWarning: Passing 'suffixes' which cause duplicate columns {'PG_x', 'PN_x', 'PL_x'} in the result is deprecated and will raise a MergeError in a future version.
  result_df = pd.merge(result_df, category_relative_percentages.reset_index(


,state,mobility_type,x_time,x_date,y_ridership_change,phase,t4_1pct_5days,t4_1pct_7days,t4_1pct_14days,t4_5pct_5days,t4_5pct_7days,t4_5pct_14days,t4_1pct_5days_category,t4_1pct_7days_category,t4_1pct_14days_category,t4_5pct_5days_category,t4_5pct_7days_category,t4_5pct_14days_category
0,Alabama,grocery_and_pharmacy_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.454906205, -...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,NaN,NaN,NaN,PN,PN,PN,PN,PN,PN
1,Alabama,parks_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.227272727, 5....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",96.0,NaN,NaN,87.0,87.0,87.0,PG,PN,PN,PG,PG,PG
2,Alabama,residential_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.187229437, 1....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",93.0,93.0,NaN,75.0,75.0,75.0,PG,PG,PN,PG,PG,PG
3,Alabama,retail_and_recreation_percent_change_from_base...,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.392135642, 1....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,81.0,81.0,81.0,PN,PN,PN,PL,PL,PL
4,Alabama,transit_stations_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.971500722, 2....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,86.0,86.0,86.0,PN,PN,PN,PL,PL,PL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Wyoming,parks_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.761904762, 6....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",159.0,159.0,159.0,75.0,75.0,75.0,PG,PG,PG,PG,PG,PG
308,Wyoming,residential_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.80952381, 0.7...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,75.0,75.0,75.0,PN,PN,PN,PG,PG,PG
309,Wyoming,retail_and_recreation_percent_change_from_base...,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.619047619, 3....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,75.0,75.0,NaN,PN,PN,PN,PL,PL,PN
310,Wyoming,transit_stations_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.142857143, 5....","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,75.0,75.0,120.0,PN,PN,PN,PL,PL,PG


### MIN MAX and cap data so there is no inf value. Change Cap Value if needed. This method is work in progress as it is assuming quite a few factors. Feel free to change.

In [2]:
# Compute min max and caps


import numpy as np
import pandas as pd  # Assuming you're using pandas for your dataset


dataset1 = dataset


def get_global_min_max(dataset, column_name):
    all_values = [value for sublist in dataset[column_name].tolist()
                  for value in sublist]
    return min(all_values), max(all_values)


def cap_values(lst, cap_value=-1):
    # Convert numpy array to list if necessary
    lst_list = lst.tolist() if isinstance(lst, np.ndarray) else lst

    # Start from the minimum of the list
    min_index = 30

    # Check the number of values to the right of the minimum value
    num_values_to_right = len(lst_list) - min_index - 1

    if num_values_to_right >= 10:
        # Cap values above the cap_value to be -cap_value
        return [value if value < cap_value else cap_value for value in lst_list]
    else:
        # Return the original list
        return [value if value > cap_value else cap_value for value in lst_list]


def compute_derivative(lst):
    return np.append([0], np.gradient(lst))


def minmax_normalize(lst, min_val, max_val):
    return [(x - min_val) / (0 - min_val) for x in lst]


# Cap values in 'smoothed' to a minimum absolute value of 0.05
#dataset['smoothed'] = dataset['smoothed'].apply(cap_values)
dataset['y_ridership_change'] = dataset['y_ridership_change'].apply(cap_values)

# Compute the derivative of the capped values
'''dataset['derivative_smoothed'] = dataset['smoothed'].apply(
    compute_derivative)
dataset['derivative_ridership_change'] = dataset['y_ridership_change'].apply(
    compute_derivative)'''

# If you want to drop rows with NaN values in the selected column

# Min-max normalization (commented out)

'''global_min_smoothed, global_max_smoothed = get_global_min_max(
    dataset, 'smoothed')'''
global_min_change, global_max_change = get_global_min_max(
    dataset, 'y_ridership_change')

'''dataset['smoothed'] = dataset['smoothed'].apply(
    minmax_normalize, args=(global_min_smoothed, global_max_smoothed))
dataset['derivative_smoothed'] = dataset['smoothed'].apply(
    compute_derivative)'''
dataset['y_ridership_change'] = dataset['y_ridership_change'].apply(
    minmax_normalize, args=(global_min_change, global_max_change))


print(len(dataset1))


'''plt.plot(dataset['derivative_smoothed'][2])
plt.plot(dataset['smoothed'][2])'''
'''dataset1 = dataset1[dataset1['mobility_type'] ==
                    'workplaces_percent_change_from_baseline']'''
dataset1


312


,state,mobility_type,x_time,x_date,y_ridership_change,phase,t4_1pct_5days,t4_1pct_7days,t4_1pct_14days,t4_5pct_5days,t4_5pct_7days,t4_5pct_14days,t4_1pct_5days_category,t4_1pct_7days_category,t4_1pct_14days_category,t4_5pct_5days_category,t4_5pct_7days_category,t4_5pct_14days_category
0,Alabama,grocery_and_pharmacy_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,NaN,NaN,NaN,PN,PN,PN,PN,PN,PN
1,Alabama,parks_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",96.0,NaN,NaN,87.0,87.0,87.0,PG,PN,PN,PG,PG,PG
2,Alabama,residential_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",93.0,93.0,NaN,75.0,75.0,75.0,PG,PG,PN,PG,PG,PG
3,Alabama,retail_and_recreation_percent_change_from_base...,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,81.0,81.0,81.0,PN,PN,PN,PL,PL,PL
4,Alabama,transit_stations_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,86.0,86.0,86.0,PN,PN,PN,PL,PL,PL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Wyoming,parks_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",159.0,159.0,159.0,75.0,75.0,75.0,PG,PG,PG,PG,PG,PG
308,Wyoming,residential_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,75.0,75.0,75.0,PN,PN,PN,PG,PG,PG
309,Wyoming,retail_and_recreation_percent_change_from_base...,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,75.0,75.0,NaN,PN,PN,PN,PL,PL,PN
310,Wyoming,transit_stations_percent_change_from_baseline,"[44972, 44973, 44974, 44975, 44976, 44977, 449...","[15-Feb, 16-Feb, 17-Feb, 18-Feb, 19-Feb, 20-Fe...","[0.987115773973152, 0.987115773973152, 0.98711...","[D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, ...",NaN,NaN,NaN,75.0,75.0,120.0,PN,PN,PN,PL,PL,PG


### h(t), f(t), F(t), S(t) calculation

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score  # Import r2_score
from sklearn.metrics import mean_squared_error, r2_score

Qmax_values = {}
alpha_values = {}
beta_values = {}
delta_values = {}
initial_values = {}
df = pd.DataFrame()
res = []

df_long_format = pd.DataFrame(columns=['state', 'mobility_type'])

t4_cutoff = 't4_5pct_7days'
dataset1 = dataset[dataset[f'{t4_cutoff}_category'] == 'PL']

mobility_types = dataset1['mobility_type'].unique()

for mobility_type in mobility_types:
    dataset_filtered = dataset1[dataset1['mobility_type']
                                == mobility_type].reset_index(drop=True)

    for i in range(len(dataset_filtered)):
        state = dataset_filtered['state'][i]
        x_date = np.array(dataset_filtered['x_date'][i])
        queue_data = np.array(dataset_filtered['y_ridership_change'][i])

        time_index_of_min = np.argmin(queue_data)

        t4 = int(dataset_filtered[t4_cutoff][i])+1
        t2 = int(time_index_of_min)-1

        queue_data = queue_data[:t4+1]

        Qmax = queue_data[t2]

        initial = queue_data[1]
        Delta = queue_data[-1]
        alpha = (-6 * Qmax) / (t2 ** 3)
        beta = (6 * (Qmax - Delta)) / ((t4 - t2) ** 3)
        # Calculate Qmax, alpha, beta, etc. using modified_deviation

        Qmax_values[(mobility_type, i)] = Qmax
        alpha_values[(mobility_type, i)] = alpha
        beta_values[(mobility_type, i)] = beta
        initial_values[(mobility_type, i)] = initial
        delta_values[(mobility_type, i)] = Delta

        h_t_prefix = np.full(t2, np.nan)
        t = np.array(range(len(queue_data[t2:t4+1])))

        # Original Data
        F_t_suffix = np.array(queue_data[t2:t4+1])
        F_t = np.concatenate((h_t_prefix, F_t_suffix))

        # S_t calculation
        S_t_suffix = 1 - F_t_suffix
        S_t = np.concatenate((h_t_prefix, S_t_suffix))

        # Derivative (Gradient)
        f_t_suffix = np.gradient(F_t_suffix)
        f_t = np.concatenate((h_t_prefix, f_t_suffix))

        # Calculate h_t
        h_t_suffix = f_t_suffix / (1 - F_t_suffix)
        h_t = np.concatenate((h_t_prefix, h_t_suffix))

        # 1. Polynomial Fit
        coeffs = np.polyfit(t, f_t_suffix, 2)
        polynomial_fit = np.poly1d(coeffs)
        fitted_f_t_poly_suffix = polynomial_fit(t)
        fitted_f_t_poly = np.concatenate((h_t_prefix, fitted_f_t_poly_suffix))

        # Calculate h_t using the polynomial fitted f_t
        h_t_poly_suffix = fitted_f_t_poly_suffix / (1 - F_t_suffix)
        h_t_poly = np.concatenate((h_t_prefix, h_t_poly_suffix))

        # 2. Moving Average
        f_t_series = pd.Series(f_t_suffix)
        fitted_f_t_ma_suffix = f_t_series.rolling(
            window=7, min_periods=1).mean().values
        fitted_f_t_ma = np.concatenate((h_t_prefix, fitted_f_t_ma_suffix))

        # Calculate h_t using the moving average fitted f_t
        h_t_ma_suffix = fitted_f_t_ma_suffix / (1 - F_t_suffix)
        h_t_ma = np.concatenate((h_t_prefix, h_t_ma_suffix))

        # Padding to 250
        num_zeros_to_append = 250 - len(h_t)
        h_t = np.concatenate((h_t, np.full(num_zeros_to_append, np.nan)))
        f_t = np.concatenate((f_t, np.full(num_zeros_to_append, np.nan)))
        S_t = np.concatenate((S_t, np.full(num_zeros_to_append, np.nan)))
        F_t = np.concatenate((F_t, np.full(num_zeros_to_append, np.nan)))

        fitted_f_t_poly = np.concatenate(
            (fitted_f_t_poly, np.full(num_zeros_to_append, np.nan)))
        h_t_poly = np.concatenate(
            (h_t_poly, np.full(num_zeros_to_append, np.nan)))
        fitted_f_t_ma = np.concatenate(
            (fitted_f_t_ma, np.full(num_zeros_to_append, np.nan)))
        h_t_ma = np.concatenate((h_t_ma, np.full(num_zeros_to_append, np.nan)))

        # Create DataFrame
        df = pd.DataFrame({
            'state': [state] * 250,
            'mobility_type': [mobility_type] * 250,
            "t": np.arange(250),
            'h(t)': h_t,
            'f(t)': f_t,
            'S(t)': S_t,
            'F(t)': F_t,
            'f(t)_poly': fitted_f_t_poly,
            'h(t)_poly': h_t_poly,
            'f(t)_ma': fitted_f_t_ma,
            'h(t)_ma': h_t_ma,
            'Qmax': [Qmax] * 250,
            'Delta': [Delta] * 250
        })

        # Append the temporary DataFrame to the main long format DataFrame
        df_long_format = pd.concat(
            [df_long_format, df], ignore_index=True)


df_long_format.dropna(inplace=True)

df_long_format.to_csv(
    f'Results/Calculation Results/calculation_results_{t4_cutoff}.csv')
